# **Hercoderen gemeenten 2023**

In 2023 worder er een drie gemeenten samengevoegd tot een nieuwe gemeent. Dit werkt door op een aantal plaatsen in de database:

1. In de definitie van de PROTOS, er moeten voor de nieuwe gemeente Protos worden aangemaakt en voor de oude gemeenten niet meer.
2. In de data voor de Rioolwaterzuiveringsinstallaties (RWZI's). Waarschijnlijk wordt dit gewijzigd aangeleverd door RIVM, maar voor de zekerheid is er code ontwikkeld om dit op te vangen mocht de data in de oude indeling worden aangeleverd.
3. In de Static tabellen met bevolkingsaantallen. Dit start met de tabel Population\_Base en daarvan worden andere Static-tabellen afgeleid. In de Stored Procedure die de data van Stage naar Inter kopieert, wordt de nieuwe gemeentelijke indeling aangebracht m.b.v. een mapping-tabel. Alle tabellen die afhankelijk zijn van VWSInter.Population\_Base zullen nu ook de nieuwe gemeentelijke indeling krijgen. 
4. In één tabel wordt bevolkingsgegevens per gemeente ingelezen in het VWSStage-schema. De stored-procedure die de data kopieert naar VWSStatic worden aangepast om de data in de nieuwe indeling in VWSStatic te schrijven op dezelfde manier als dat voor de Population\_Base tabel gebeurt. 
5. Een tabel met gemeentelijke bevolkingsaantallen wordt direct ingelezen in het VWSStatic schema. Dit wordt aangepast door deze eerst in het VWSStage schema in te lezen en dan middels een Stored Procedure de gegevens in de juiste indeling in het VWSStatic schema te schrijven (zelfde werkwijze als bij 3 en 4). 
6. In één van de views wordt gerefereerd aan de tabel Safety\_Regions\_per\_Municipal. Hierin staat de relatie tussen gemeenten en veiligheidsregios. Toegevoegd wordt dat alleen de meest recente dataset met deze relatie wordt gebruikt, zodat na het inlezen van een dataset met de nieuwe gemeentelijke indeling, deze view ook direct goed staat.
7. In de tabel VWSSTATIC.Active\_Municipalities staat een lijst met alle actieve gemeenten. Hier moet een nieuwe set aan worden toegevoegd waarin de drie 'oude' gemeenten niet meer voorkomen en waaraan Voorne aan Zee is toegevoegd als nieuwe gemeente. 

> N.B. voor 6 moet dan wel een nieuw bestand met Safety\_Regions\_per\_Municipal worden klaargezet in ingelezen. 

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
    ]
}
```

## **1    Aanpassen Protos**

In het notebook MunicipalityReclassification_2023 wordt de proto voor de nieuwe gemeente aangemaakt.

## **2    Aanpassingen voor RWZI's**

Hier wordt gebruik gemaakt van de al aanwezige stored procedure **SP\_MERGE\_MOVE\_STATIC\_CBS\_RWZI** om wijzigingen aan te brengen in de mapping van RWZI's naar gemeenten. Hiervoor wordt er een aantal regels toegevoegd aan het csv-bestand "**Aantal\_inwoners\_per\_verzorgingsgebied\_mutaties**.csv" in de blob-storage, Datafiles\\Static map. 

Inhoud van deze csv wordt dan:

rwzi\_code;rwzi\_naam;startdatum;einddatum;inwoners;regio\_type;regio\_code;regio\_naam;aandeel;toelichting

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM1740;Neder-Betuwe;1.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM1740;Neder-Betuwe;0.0%;definitief

9043;LIENDEN;01-01-2020;07-12-2020;6728;GM;GM0214;Buren;98.5%;definitief

9043;LIENDEN;07-12-2020;31-12-2020;0;GM;GM0214;Buren;0.0%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0214;Buren;5.2%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0214;Buren;15.3%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM0281;Tiel;75.3%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM0281;Tiel;66.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1740;Neder-Betuwe;13.7%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1740;Neder-Betuwe;12.7%;definitief

9025;TIEL;01-01-2020;07-12-2020;56015;GM;GM1960;West Betuwe;5.9%;definitief

9025;TIEL;07-12-2020;31-12-2020;62813;GM;GM1960;West Betuwe;5.3%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0297;Zaltbommel;97.8%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0297;Zaltbommel;0.0%;definitief

9029;AALST;01-01-2020;05-10-2020;10495;GM;GM0263;Maasdriel;2.2%;definitief

9029;AALST;05-10-2020;31-12-2020;0;GM;GM0263;Maasdriel;0.0%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0263;Maasdriel;55.8%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0263;Maasdriel;45.1%;definitief

9041;ZALTBOMMEL;01-01-2020;05-10-2020;41974;GM;GM0297;Zaltbommel;44.2%;definitief

9041;ZALTBOMMEL;05-10-2020;31-12-2020;53462;GM;GM0297;Zaltbommel;54.9%;definitief

12014;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0370;Beemster;0.0%;definitief

12041;BEEMSTER;01-01-2020;31-12-9999;104382;GM;GM0439;Purmerend;9.7%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0398;Heerhugowaard;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM0416;Langedijk;0%;definitief

12013;GEESTMERAMBACHT;01-01-2020;31-12-9999;185792;GM;GM1980;Dijk en Waard;46.7%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0756;Boxmeer;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0786;Grave;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0815;Mill en Sint Hubert;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1684;Cuijk;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1702;Sint Anthonis;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1982;Land van Cuijk;6.91%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1982;Land van Cuijk;90.07%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM0856;Uden;0%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1685;Landerd;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM0856;Uden;0%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1685;Landerd;0%;definitief

28005;DINTHER;01-01-2020;31-12-9999;123719;GM;GM1991;Maashorst;32.60%;definitief

29002;OIJEN;01-01-2020;31-12-9999;160856;GM;GM1991;Maashorst;5.64%;definitief

29003;LAND VAN CUIJK (HAPS);01-01-2020;31-12-9999;87802;GM;GM1991;Maashorst;9.91%;definitief

## **3    Aanpassen bevolkingsaantallen Population\_Base**

Om de gemeentelijke indeling te kunnen aanpassen naar de gewenste indeling wordt er een mapping tabel gemaakt die de wijzigingen bevat. Met behulp van deze mapping-tabel wordt de data die van VWSTAGE naar VWSSTATIC wordt gekopieerd, direct in de goede gemeentelijke indeling gezet. Dit gebeurt in de Stored Procedure **SP\_CBS\_POPULATION\_BASE\_STATIC**.

N.B. de kolom "Active" in de tabel RECLASSIFIED_MUNICIPALITY_MAPPING staat default op 0, wat betekent dat deze grenswijzigingen _NIET_ worden uitgevoerd. Om ze actief te maken moet met een update commando de waarde in de kolom "Active" op 1 worden gezet: 

Update VWSStatic.RECLASSIFIED_MUNICIPALITY_MAPPING SET Active = 1

FROM VWSStatic.RECLASSIFIED_MUNICIPALITY_MAPPING 

WHERE Date\_Last\_Inserted = (SELECT Max(Date\_Last\_Inserted) )

In [ ]:
/*
	Hier wordt een tabel gemaakt waarin informatie wordt
	opgeslagen omtrent herindelingen van gemeenten. 
*/

DROP TABLE IF EXISTS VWSSTATIC.RECLASSIFIED_MUNICIPALITY_MAPPING

CREATE TABLE VWSSTATIC.RECLASSIFIED_MUNICIPALITY_MAPPING
(
	[ID] [int] IDENTITY(1,1) NOT NULL,
	[DATE_LAST_INSERTED] [datetime] NOT NULL Default GetDate(),
	[GM_CODE_ORIGINAL] [nvarchar](10) NULL,
	[GM_NAME_ORIGINAL] [nvarchar](255) NULL,
	[GM_CODE_NEW] [nvarchar](10) NULL,
	[GM_NAME_NEW] [nvarchar](255) NULL,
	[VR_CODE] [nvarchar](10) NULL,
	[VR_NAME] [nvarchar](255) NULL,
	[PROVINCE_CODE] [nvarchar](10) NULL,
	[PROVINCE_NAME] [nvarchar](255) NULL,
	[GGD_CODE] [nvarchar](10) NULL,
	[GGD_NAME] [nvarchar](255) NULL,
	[SHARES] [decimal](16, 6) NOT NULL,
	[ACTIVE] [int] NOT NULL Default(0)
)
--457
--
INSERT INTO VWSSTATIC.RECLASSIFIED_MUNICIPALITY_MAPPING (GM_CODE_ORIGINAL,	[GM_NAME_ORIGINAL],GM_CODE_NEW,GM_NAME_NEW,VR_CODE, VR_NAME,PROVINCE_CODE,PROVINCE_NAME,GGD_CODE,GGD_NAME,SHARES,ACTIVE)
VALUES ('GM1684','Cuijk','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
     , ('GM0756','Boxmeer','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM0815','Mill en Sint Hubert','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM1702','Sint Anthonis','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM0786','Grave','GM1982','Land van Cuijk','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM1685','Landerd','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM0856','Uden','GM1991','Maashorst','VR21','Brabant-Noord','PV30','Noord-Brabant','GG5406','GGD Hart voor Brabant',1,1)
	 , ('GM0398','Heerhugowaard','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,1)
	 , ('GM0416','Langedijk','GM1980','Dijk en Waard','VR10','Noord-Holland-Noord','PV27','Noord-Holland','GG2707','GGD Hollands-Noorden',1,1)
	 , ('GM0370','Beemster','GM0439','Purmerend','VR11','Zaanstreek-Waterland','PV27','Noord-Holland','GG7306','GGD Zaanstreek/Waterland',1,1)
	 , ('GM0501','Brielle','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,1)
	 , ('GM0530','Hellevoetsluis','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,1)
	 , ('GM0614','Westvoorne','GM1992','Voorne aan Zee','VR17','Rotterdam-Rijnmond','PV28','Zuid-Holland','GG4607','GGD Rotterdam-Rijnmond',1,1)
	 , ('GM0457','Weesp','GM0363','Amsterdam','VR13','Amsterdam-Amstellan','PV27','Noord-Holland','GG3406','GGD Amsterdam',1,1)
GO

